In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def filter_rmats(infile,outfile,newoutfile,listsamples,FDRval,deltaval,mean_cov,filter_events):
    # Load the CSV file
    file_path = infile
    data = pd.read_csv(file_path, delimiter='\t')
    file = open(file_path,'r')
    
    output = open(outfile,'w')
    output_new = open(newoutfile,'w')
    COLUMN_NAMES = listsamples
    #COLUMN_NAMES = ['RNU.23D2296','RNU.24D0834','RNUR.24D0921','RNUR.24D0945','RNU.24D0986','RNUR.24D0987','RNUR.24D0988','RNU.24D0989','RNUR.24D0991','RNU.24D0992','RNUR.24D1039','RNUR.24D1108','CTR.24D0873','CTR.24D0654','CTR.24D0971','CTR.24D0679','CTR.23D1450','CTR.23D0981','CTR.23D1002','CTR.23D1432','CTR.23D1576','CTR.23D0984','CTR.23D1172','CTR.23D2426','CTR.23D2492','CTR.23D2306','CTR.23D2600','CTR.23D2300','CTR.23D2488','CTR.24D0192','CTR.23D2604','CTR.23D2423','CTR.23D2442','CTR.23D2504','CTR.23D2505','CTR.23D2539','CTR.23D2449','CTR.22D2375','CTR.23D0699','CTR.23D0357','CTR.23D0686','CTR.23D0243','CTR.23D0676','CTR.23D0757','CTR.23D0674','CTR.23D2003','CTR.23D2176','CTR.23D2149','CTR.23D2245','CTR.23D2051','CTR.23D2003','CTR.23D0244','CTR.22D1947','CTR.22D2197','CTR.23D0122','CTR.23D1029','CTR.23D1748','CTR.23D1761','CTR.23D2606','CTR.23D2671','CTR.24D0420','CTR.23D2624','CTR.24D0157','CTR.24D0213','CTR.23D2655','CTR.23D0946','CTR.23D0855','CTR.23D0838','CTR.23D1955','CTR.23D1808','CTR.23D1908','CTR.23D1937','CTR.23D1700','CTR.23D1945','CTR.24D0484','CTR.24D0451','CTR.24D0380','CTR.24D0334','CTR.24D0467','CTR.24D0406','CTR.24D0438','CTR.24D0593','CTR.24D0224','CTR.24D0439','CTR.24D0428','CTR.24D0434','CTR.24D0471']
    out_F = 0
    in_F = 0
    sig_genes = {}
    sig_genes_key = {}
    
    df = pd.DataFrame(columns=COLUMN_NAMES)
    df_new = pd.DataFrame(columns=COLUMN_NAMES)
    events = []
    a = 0
    for i in file:
        a+=1
        if a == 1:
            output.write(i)
            output_new.write(i)
        else:
            field = i.split('\t')
            tag = True
            IJC = [int(x) for x in field[12].split(',')]
            SJC = [int(x) for x in field[13].split(',')]
            IJC2 = [int(x) for x in field[14].split(',')]
            SJC2 = [int(x) for x in field[15].split(',')]
            FDR = float(field[19])
            psi1 = [float(x) for x in field[20].split(',') if x != 'NA']
            psi2 = [float(x) for x in field[21].split(',') if x != 'NA']
            psitot = psi1 + psi2
            delta = float(field[22].rstrip())
            geneSymbol = field[2].replace('\"','')
    
            if FDR > FDRval or (delta > -deltaval and delta < deltaval):
                tag = False
            
            #if (np.mean(IJC) < 10 and np.mean(IJC2) < 10) or (np.mean(SJC) < 10 and np.mean(SJC2) < 10):
            #    tag = False
            for j in IJC,SJC:
                if np.mean(j) < mean_cov:
                    tag = False                
            
            if tag:
                #check if event is out or inframe
                if field[4] == '+':
                    lenght = int(field[6]) - int(field[8])
                    if float(lenght) % 3 == 0:
                        in_F += 1
                    else:
                        out_F +=1
                else:
                    lenght = int(field[7]) - int(field[5])
                    if float(lenght) % 3 == 0:
                        in_F += 1
                    else:
                        out_F +=1
                        
                if filter_events:
                    if field[3]+'-'+field[5]+'-'+field[6]+'-'+field[7]+'-'+field[8] not in filter_events:
                        continue
                
                events.append(field[3]+'-'+field[5]+'-'+field[6]+'-'+field[7]+'-'+field[8])
                output.write(i)
                
                if (np.mean(IJC2) > 10 and np.mean(SJC2) < 3) or (np.mean(IJC) > 10 and np.mean(SJC) < 3):
                    output_new.write(i)
                    
                if len(psitot) == len(COLUMN_NAMES) and geneSymbol != 'NA':
                    if not geneSymbol in sig_genes.keys():
                        sig_genes[geneSymbol] = FDR
                        sig_genes_key[geneSymbol] = field[3]+':'+field[5]+'-'+field[6]+':'+field[7]+'-'+field[8]
                        
                        df.loc[geneSymbol] = psitot
                        if (np.mean(IJC2) > 10 and np.mean(SJC2) < 3) or (np.mean(IJC) > 10 and np.mean(SJC) < 3):
                            df_new.loc[geneSymbol] = psitot
                    else:
                        if FDR < sig_genes[geneSymbol]:
                            sig_genes[geneSymbol] = FDR
                            sig_genes_key[geneSymbol] = field[3]+':'+field[5]+'-'+field[6]+':'+field[7]+'-'+field[8]
                            df.loc[geneSymbol] = psitot
                            if (np.mean(IJC2) > 10 and np.mean(SJC2) < 3) or (np.mean(IJC) > 10 and np.mean(SJC) < 3):
                                df_new.loc[geneSymbol] = psitot
                        else:
                            continue
            
    print("Number of inframe events: "+str(in_F)+", number of out-of-frame events: "+str(out_F)+" in "+str(len(df.index))+" genes")
    
    output.close()
    output_new.close()
    file.close()

    return df, df_new, events, sig_genes_key

def PCA_df(df,listsamples,shownames,outsvg,FDR,delta):
    
    df_transposed = df.transpose()
    
    pca = PCA(n_components=2)
    pca_result = pca.fit_transform(df_transposed)
    
    y = []
    for x in listsamples:
        if 'CTR' in x:
            y.append(0)
        elif 'RNUR' in x:
            y.append(1)
        else:
            y.append(2)
    
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], c=y, edgecolor='k', s=50)
    # Annotating each point
    if shownames:
        for i, sample_name in enumerate(listsamples):
            plt.annotate(sample_name, (pca_result[i, 0], pca_result[i, 1]))
    
    #ax = plt.gca()
    #ax.set_xlim([-1, 1])
    #ax.set_ylim([-1, 1])
    
    plt.title('PCA of PSI values with FDR<'+str(FDR)+' and |deltaPSI|>'+str(delta)+', n='+str(len(df_transposed.columns)))
    plt.xlabel('Principal Component 1: '+str(int(float(pca.explained_variance_ratio_[0])*100))+'%')
    plt.ylabel('Principal Component 2: '+str(int(float(pca.explained_variance_ratio_[1])*100))+'%')
    #plt.colorbar(label='Sample Type (0=Unaffected, 1=Affected)')
    
    plt.grid(True)
    if outsvg:
        plt.savefig(outsvg, format="pdf", transparent=True)
    plt.show()

def filter_rmats_mxe(infile,outfile,newoutfile,listsamples,FDRval,deltaval,mean_cov,filter_events):
    # Load the CSV file
    file_path = infile
    data = pd.read_csv(file_path, delimiter='\t')
    file = open(file_path,'r')
    
    output = open(outfile,'w')
    output_new = open(newoutfile,'w')
    COLUMN_NAMES = listsamples
    #COLUMN_NAMES = ['RNU.23D2296','RNU.24D0834','RNUR.24D0921','RNUR.24D0945','RNU.24D0986','RNUR.24D0987','RNUR.24D0988','RNU.24D0989','RNUR.24D0991','RNU.24D0992','RNUR.24D1039','RNUR.24D1108','CTR.24D0873','CTR.24D0654','CTR.24D0971','CTR.24D0679','CTR.23D1450','CTR.23D0981','CTR.23D1002','CTR.23D1432','CTR.23D1576','CTR.23D0984','CTR.23D1172','CTR.23D2426','CTR.23D2492','CTR.23D2306','CTR.23D2600','CTR.23D2300','CTR.23D2488','CTR.24D0192','CTR.23D2604','CTR.23D2423','CTR.23D2442','CTR.23D2504','CTR.23D2505','CTR.23D2539','CTR.23D2449','CTR.22D2375','CTR.23D0699','CTR.23D0357','CTR.23D0686','CTR.23D0243','CTR.23D0676','CTR.23D0757','CTR.23D0674','CTR.23D2003','CTR.23D2176','CTR.23D2149','CTR.23D2245','CTR.23D2051','CTR.23D2003','CTR.23D0244','CTR.22D1947','CTR.22D2197','CTR.23D0122','CTR.23D1029','CTR.23D1748','CTR.23D1761','CTR.23D2606','CTR.23D2671','CTR.24D0420','CTR.23D2624','CTR.24D0157','CTR.24D0213','CTR.23D2655','CTR.23D0946','CTR.23D0855','CTR.23D0838','CTR.23D1955','CTR.23D1808','CTR.23D1908','CTR.23D1937','CTR.23D1700','CTR.23D1945','CTR.24D0484','CTR.24D0451','CTR.24D0380','CTR.24D0334','CTR.24D0467','CTR.24D0406','CTR.24D0438','CTR.24D0593','CTR.24D0224','CTR.24D0439','CTR.24D0428','CTR.24D0434','CTR.24D0471']
    out_F = 0
    in_F = 0
    sig_genes = {}
    
    df = pd.DataFrame(columns=COLUMN_NAMES)
    df_new = pd.DataFrame(columns=COLUMN_NAMES)
    events = []
    a = 0
    for i in file:
        a+=1
        if a == 1:
            output.write(i)
            output_new.write(i)
        else:
            field = i.split('\t')
            tag = True
            IJC = [int(x) for x in field[14].split(',')]
            SJC = [int(x) for x in field[15].split(',')]
            IJC2 = [int(x) for x in field[16].split(',')]
            SJC2 = [int(x) for x in field[17].split(',')]
            FDR = float(field[21])
            psi1 = [float(x) for x in field[22].split(',') if x != 'NA']
            psi2 = [float(x) for x in field[23].split(',') if x != 'NA']
            psitot = psi1 + psi2
            delta = float(field[24].rstrip())
            geneSymbol = field[2].replace('\"','')
    
            if FDR > FDRval or (delta > -deltaval and delta < deltaval):
                tag = False
            
            #if (np.mean(IJC) < 10 and np.mean(IJC2) < 10) or (np.mean(SJC) < 10 and np.mean(SJC2) < 10):
            #    tag = False
            for j in IJC,SJC:
                if np.mean(j) < mean_cov:
                    tag = False                
            
            if tag:                        
                if filter_events:
                    if field[3]+'-'+field[5]+'-'+field[6]+'-'+field[7]+'-'+field[8] not in filter_events:
                        continue
                
                events.append(field[3]+'-'+field[5]+'-'+field[6]+'-'+field[7]+'-'+field[8])
                output.write(i)
                
                if (np.mean(IJC2) > 10 and np.mean(SJC2) < 3) or (np.mean(IJC) > 10 and np.mean(SJC) < 3):
                    output_new.write(i)
                    
                if len(psitot) == len(COLUMN_NAMES):
                    if not geneSymbol in sig_genes.keys():
                        sig_genes[geneSymbol] = FDR
                        df.loc[geneSymbol] = psitot
                        if (np.mean(IJC2) > 10 and np.mean(SJC2) < 3) or (np.mean(IJC) > 10 and np.mean(SJC) < 3):
                            df_new.loc[geneSymbol] = psitot
                    else:
                        if FDR < sig_genes[geneSymbol]:
                            sig_genes[geneSymbol] = FDR
                            df.loc[geneSymbol] = psitot
                            if (np.mean(IJC2) > 10 and np.mean(SJC2) < 3) or (np.mean(IJC) > 10 and np.mean(SJC) < 3):
                                df_new.loc[geneSymbol] = psitot
                        else:
                            continue
            
    
    output.close()
    output_new.close()
    file.close()

    return df, df_new, events